In [1]:
import csv
import pip
from gensim import corpora, models, similarities
import pandas as pd
import numpy as np
import gensim
from nltk.corpus import stopwords
from scipy.spatial.distance import cosine 
import re
import dask.dataframe as dd

from utils import *

https://www.kaggle.com/puneetsl/unusual-meaning-map

### load data

In [2]:
df_train = pd.read_csv('data/train_clean_vB.csv', 
                       dtype={
                           'q1_clean_vB': np.str,
                           'q2_clean_vB': np.str
                       })



df_test = pd.read_csv('data/test_clean_vB.csv', 
                      dtype={
                          'q1_clean_vB': np.str,
                          'q2_clean_vB': np.str
                      })

In [3]:
df_train.rename(columns={"q1_clean_vB":"question1", "q2_clean_vB":"question2"}, inplace=True)
df_test.rename(columns={"q1_clean_vB":"question1", "q2_clean_vB":"question2"}, inplace=True)

In [4]:
df_train['test_id'] = -1
df_test['id'] = -1
df_test['qid1'] = -1
df_test['qid2'] = -1
df_test['is_duplicate'] = -1

df = pd.concat([df_train, df_test])
df['question1'] = df['question1'].fillna('')
df['question2'] = df['question2'].fillna('')
df['uid'] = np.arange(df.shape[0])
df = df.set_index(['uid'])

In [5]:
# df.to_csv("data/cache/dd_df_cache.csv",index=False)

In [6]:
ix_train = np.where(df['id'] >= 0)[0]
ix_test = np.where(df['id'] == -1)[0]
ix_is_dup = np.where(df['is_duplicate'] == 1)[0]
ix_not_dup = np.where(df['is_duplicate'] == 0)[0]

In [7]:
# df.ix[10].values

In [25]:
# df = dd.from_pandas(df,npartitions=8)

In [9]:
len(df)

2750086

#### embeddings

In [9]:
stops = set(stopwords.words("english"))

In [7]:
with open("data/cache/embeddings_index.npy", 'rb') as handle:
    embeddings_index = pickle.load(handle)


In [8]:
print('Word embeddings: %d' % len(embeddings_index))

Word embeddings: 2196016


In [26]:
def w2v_sim(w1, w2):
    try: 
        return cosine(embeddings_index[w1], embeddings_index[w2])
    except KeyError:
        return 0.0


In [60]:
import cv2

(30, 20)

In [70]:
def img_feature(row):
    s1 = row['question1']
    s2 = row['question2']
    
    t1 = re.split(r'\W',s1)
    t1 = filter(None, t1)
    t1 = [i for i in t1 if i not in stops]
    
    t2 = re.split(r'\W',s2)
    t2 = filter(None, t2)
    t2 = [i for i in t2 if i not in stops]  
#     print t1, t2
    
    Z = [[w2v_sim(x, y) for x in t1] for y in t2] 
    a = np.array(Z, order='C')
#     return [np.resize(a,(10,10)).flatten()]
    return [cv2.resize(a ,(10,10)).flatten()]


In [74]:
# img_feature(df.iloc[0])

In [41]:
# from tqdm import tqdm_notebook, tqdm_pandas

In [42]:
# tqdm.pandas(desc="imgage feature")

In [30]:
# df_with_image_feature = df.progress_apply(img_feature, axis=1)


In [31]:
# dmaster = dd.from_pandas(df, npartitions=8)


In [39]:
# import dask
# import dask.multiprocessing
# dask.set_options(get=dask.multiprocessing.get)

In [10]:
# img_feat = dmaster.apply(img_feature, axis=1)

In [75]:
df_with_image_feature = df.ix[0:4].apply(img_feature,axis=1)

In [76]:
# df_with_image_feature.values.reshape(5,-1)[0]

In [84]:
from multiprocessing import cpu_count,Pool
 
cores = cpu_count() #Number of CPU cores on your system
# partitions = cores-2 #Define as many partitions as you want
n_processes = cores/2

In [85]:
pool = Pool(processes=n_processes)

Process PoolWorker-9:
AttributeError: 'module' object has no attribute 'call_apply_fn'
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
  File "/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/multiprocessing/queues.py", line 378, in get
    return recv()
Process PoolWorker-10:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda2/envs/qenv/lib

In [86]:
df_split = np.array_split(df, n_processes)


In [87]:
def call_apply_fn(dft):
    return dft.apply(img_feature, axis=1)


In [ ]:
pool_results = pool.map(call_apply_fn, df_split)

In [57]:
np.array_split(df, 16)[0].shape

(171881, 7)

In [ ]:
# img_feat = parallelize(df,img_feature)

In [ ]:
1